# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import pandas as pd
import re
import string

from os import path as osp
from pprint import pprint

## Constantes e funções auxiliares

In [2]:
NULL_STRING = '#NULLO#'
FILE_ANUNCIOS = '../datasets/anuncios.parquet'
FILE_ANUNCIOS_AMOSTRA = '../datasets/anuncios_amostra.parquet'

In [3]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [4]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [5]:
def extract_magalu_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

In [6]:
def format_issue(row,remove_punct=False):
    nome,desc,img0,img1,img2 = row
    
    # remove null string
    desc = desc.replace(NULL_STRING,'')
    # remove punctiation from nome and desc
    if remove_punct:
        nome = nome.translate(str.maketrans('', '', string.punctuation))
        desc = desc.translate(str.maketrans('', '', string.punctuation))

    # remove ponto-e-vírgula e aspas duplas para evitar a 
    # identificação incorreta de campos ao importar para o redmine
    desc = desc.replace(';',',')
    desc = desc.replace('"',"''")
    # remove multiple linebrakes and spaces
    desc = re.sub('\n+','\n\n',desc)
    desc = re.sub('\s+',' ',desc)
    desc = desc.strip()
    
    img_css = '{max-width:250px; height:auto}'
    if len(img2) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}! !{img_css}{img2}!'
    elif len(img1) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}!'
    elif len(img0) > 0:
        img_list = f'!{img_css}{img0}!'
    else:
        img_list = ''
        
    if len(desc) >= 1:
        issue = f'h1. {nome}\n\n{desc}\n\n{img_list}'
    else:
        issue = f'h1. {nome}\n\n{img_list}'
    
    return issue.strip()

# Carga e prepação dos dados

## Dados pré processados

In [7]:
if osp.exists(FILE_ANUNCIOS):
    df_anuncios = pd.read_parquet(FILE_ANUNCIOS)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


## Dados brutos

### Amazon

In [8]:
file_amazon_reindexed = '../datasets/amazon_reindexed.parquet'
df_amazon = pd.read_parquet(file_amazon_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Descrição_Principal', 'Imagem', 'Imagens' ]
df_amazon = df_amazon[columns_to_keep]

df_amazon['desc'] = df_amazon[['Descrição', 'Descrição_Principal']].apply(lambda row: extract_amazon_desc(row),axis=1)
# df_amazon['img_list'] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)
df_amazon[['img0','img1','img2']] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)

columns_to_keep = ['Link', 'Nome', 'desc', 'img0','img1','img2']
df_amazon = df_amazon[columns_to_keep]
df_amazon.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']
df_amazon.head()

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


### Magalu

In [9]:
file_magalu_reindexed = '../datasets/magalu_redindexed.parquet.gzip'
df_magalu = pd.read_parquet(file_magalu_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_magalu = df_magalu[columns_to_keep]

df_magalu[['img0','img1','img2']] = df_magalu[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_magalu = df_magalu[columns_to_keep]
df_magalu.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_magalu.sample(5)

,link,nome,desc,img0,img1,img2
30198,https://www.magazineluiza.com.br/carregador-po...,Carregador Power Bank 10000 mAh Com Cabo V8 Co...,Introduzindo o Carregador Power Bank de 10000 ...,https://a-static.mlcdn.com.br/280x210/carregad...,https://a-static.mlcdn.com.br/90x90/carregador...,
17173,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,,,
11087,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/case-sig...,,
53418,https://www.magazineluiza.com.br/smartphone-mo...,Smartphone Moto G53 128GB 4G 5G Wi-Fi Tela 6.5...,O único jeito de usar é o seu.Velocidade 5G*. ...,https://a-static.mlcdn.com.br/280x210/smartpho...,https://a-static.mlcdn.com.br/90x90/smartphone...,https://a-static.mlcdn.com.br/90x90/smartphone...
53775,https://www.magazineluiza.com.br/smartphone-mo...,Smartphone Motorola Moto E30 XT2159-2 Dual 32 ...,Smartphone Motorola Moto E30 XT2159-2 Dual 32 ...,https://a-static.mlcdn.com.br/90x90/smartphone...,,


### Mercado Livre

In [10]:
file_ml_reindexed = '../datasets/ml_redindexed.parquet.gzip'
df_ml = pd.read_parquet(file_ml_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_ml = df_ml[columns_to_keep]

df_ml[['img0','img1','img2']] = df_ml[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_ml = df_ml[columns_to_keep]
df_ml.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_ml.sample(5)

,link,nome,desc,img0,img1,img2
72003,https://www.mercadolivre.com.br/projetor-wewat...,Projetor Wewatch V50 5g 1080p Wifi/bluetooth 2...,Obtenha clareza e resolução exclusivas nas sua...,https://http2.mlstatic.com/D_NQ_NP_866731-MLB5...,https://http2.mlstatic.com/D_NQ_NP_866731-MLB5...,
26062,https://produto.mercadolivre.com.br/MLB-329153...,"Smartphone Samsung Galaxy A34 5g Violeta, 128g...",OFERTA EXCLUSIVA PARA COMPRA NO MERCADO LIVRE ...,https://http2.mlstatic.com/D_NQ_NP_907753-MLB6...,,
20508,https://produto.mercadolivre.com.br/MLB-310288...,"Localizador De Sinais Digitais De Satélite, Mi...",Características: Medidor de força do sinal IF ...,https://http2.mlstatic.com/D_NQ_NP_719120-CBT5...,,
71359,https://www.mercadolivre.com.br/notebook-gamer...,Notebook gamer Acer Aspire Nitro 5 AN515-54 p...,Um novo nível de jogo é possível com este note...,https://http2.mlstatic.com/D_NQ_NP_627861-MLA4...,,
53210,https://produto.mercadolivre.com.br/MLB-402378...,Kit Inova Tv Box 4k Dig-7021 4k 512gb Memória ...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_992636-MLB7...,,


### União dos dados em único dataframe

In [11]:
df_anuncios = pd.concat([df_amazon, df_magalu, df_ml])
df_anuncios = df_anuncios.drop_duplicates(subset='nome')
df_anuncios = df_anuncios.reset_index(drop=True)
df_anuncios.to_parquet(FILE_ANUNCIOS)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


# Selecionar e preparar amostra

## Selecionar amostra

In [12]:
n_samples = 20000
df_samples_to_label = df_anuncios.iloc[:,1:].sample(n_samples,random_state=111)
df_samples_to_label.head()

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,,,
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,,,
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,,,
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_964769-MLB4...,,
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",https://http2.mlstatic.com/D_NQ_NP_916585-MLB5...,,


## Exportar para LabelStudio

In [13]:
df_label_studio = df_samples_to_label.copy()
df_label_studio.head(5)

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,,,
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,,,
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,,,
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_964769-MLB4...,,
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",https://http2.mlstatic.com/D_NQ_NP_916585-MLB5...,,


In [14]:
# Lista de imagens para baixar
images = []
images.extend(df_label_studio['img0'].to_list())
images.extend(df_label_studio['img1'].to_list())
images.extend(df_label_studio['img2'].to_list())
images = [f'wget {image}\n' for image in images if len(image)>0]
with open('images_to_download.sh','w') as f:
    f.writelines(images)
images[:5]

['wget https://http2.mlstatic.com/D_NQ_NP_964769-MLB47511585800_092021-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_916585-MLB50832387072_072022-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_933918-MLB73838529583_012024-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_939283-MLB71022987135_082023-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_762629-MLB71451841056_092023-F.webp\n']

In [15]:
# ajustar caminho das imagens (local storage)
for col in ['img0', 'img1', 'img2']:
    df_label_studio[col] = df_label_studio[col].apply(lambda x: f'/data/local-files/?d=images/{x.split("/")[-1]}' if len(x) > 0 else '/data/local-files/?d=images/none.jpg')
df_label_studio.head()

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,/data/local-files/?d=images/D_NQ_NP_964769-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",/data/local-files/?d=images/D_NQ_NP_916585-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [16]:
if 'sample_id' not in df_label_studio.columns:
    print('Create sample_id column')
    df_label_studio = df_label_studio.reset_index()
    new_columns = ['sample_id', 'nome', 'desc', 'img0', 'img1', 'img2']
    df_label_studio.columns = new_columns
df_label_studio.head()

Create sample_id column


,sample_id,nome,desc,img0,img1,img2
0,114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
1,61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
2,101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
3,181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,/data/local-files/?d=images/D_NQ_NP_964769-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
4,177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",/data/local-files/?d=images/D_NQ_NP_916585-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [17]:
samples = df_label_studio.to_dict('records')
samples[0]
with open('../datasets/samples_to_label_studio.json','w') as f:
    json.dump(samples,f)
pprint(samples[0])

{'desc': ' ',
 'img0': '/data/local-files/?d=images/none.jpg',
 'img1': '/data/local-files/?d=images/none.jpg',
 'img2': '/data/local-files/?d=images/none.jpg',
 'nome': 'Homoyoyo Prateleira De Suporte 3 Peças Suporta Suporte Para Laptop '
         'Montagem Para Laptop Suporte De Mesa Para Laptop Suportes De '
         'Prateleira Para Monitor Prateleira Para Tv Montagem Em',
 'sample_id': 114007}


## Exportar para fiscaliza

In [18]:
fiscaliza_sample_size = 400

fiscaliza_last_issue = 0
format_title = lambda i: f'LABEL_PACP_2024_{str(i+1+fiscaliza_last_issue).zfill(8)}'

df_fiscaliza = df_samples_to_label.copy()
df_fiscaliza = df_fiscaliza.sample(fiscaliza_sample_size,random_state=1653)
df_fiscaliza = df_fiscaliza.reset_index(drop=True)

columns_to_issue = ['nome', 'desc', 'img0', 'img1', 'img2']

df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)
df_fiscaliza['Descrição'] = df_fiscaliza[columns_to_issue].apply(lambda row: format_issue(row),axis=1)

df_fiscaliza['Unidade Descentralizada'] = 'UO021'
df_fiscaliza['Data de início'] = '09/04/2024'
df_fiscaliza['Data limite'] = '31/05/2024'

fiscais = ['maxwelfreitas', 
           'costalo',
           'josias',
           'romildo',
           'williams',
           'maxwelfreitas', 
           'joaotavares', 
           'peluchi']

#specify number of rows in each chunk
n = len(fiscais)
n_chuncks = round(len(df_fiscaliza)/n)

#split DataFrame into chunks
list_chuncks = [df_fiscaliza[i:i+n_chuncks].index for i in range(0,len(df_fiscaliza),n_chuncks)]
for i,chunck in enumerate(list_chuncks):
    df_fiscaliza.loc[chunck,'Atribuído para'] = fiscais[i]

columns_to_issues = ['Título', 'Descrição', 'Unidade Descentralizada', 'Data de início', 'Data limite', 'Atribuído para']

file_samples_fiscaliza = '../datasets/samples_to_fiscaliza.csv'
file_samples_fiscaliza_issues = '../datasets/samples_to_fiscaliza_issues.csv'

df_fiscaliza.to_csv(file_samples_fiscaliza,index=False,sep=';')
df_fiscaliza[columns_to_issues].to_csv(file_samples_fiscaliza_issues,index=False,sep=';')

df_fiscaliza.head()

,nome,desc,img0,img1,img2,Título,Descrição,Unidade Descentralizada,Data de início,Data limite,Atribuído para
0,Bediffer Protetor de tensão de drone RC de 16V...,,,,,LABEL_PACP_2024_00000001,h1. Bediffer Protetor de tensão de drone RC de...,UO021,09/04/2024,31/05/2024,maxwelfreitas
1,DEERC Drone with Camera for Adults and Kids 2K...,,https://m.media-amazon.com/images/I/41yOKN+NWg...,https://m.media-amazon.com/images/I/51v1nospF1...,https://m.media-amazon.com/images/I/51SHh8-Se+...,LABEL_PACP_2024_00000002,h1. DEERC Drone with Camera for Adults and Kid...,UO021,09/04/2024,31/05/2024,maxwelfreitas
2,"Cabeceira para cama box Solteiro 0,90 Cancun -...",Diferencias Cabeceira cama box : Design Modern...,https://a-static.mlcdn.com.br/90x90/cabeceira-...,https://a-static.mlcdn.com.br/90x90/cabeceira-...,https://a-static.mlcdn.com.br/90x90/cabeceira-...,LABEL_PACP_2024_00000003,"h1. Cabeceira para cama box Solteiro 0,90 Canc...",UO021,09/04/2024,31/05/2024,maxwelfreitas
3,Carregador Sem Fio Magsafe Para iPhone Watch A...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_728388-MLB5...,,,LABEL_PACP_2024_00000004,h1. Carregador Sem Fio Magsafe Para iPhone Wat...,UO021,09/04/2024,31/05/2024,maxwelfreitas
4,"Carregador de painel solar, carregador solar d...",,,,,LABEL_PACP_2024_00000005,"h1. Carregador de painel solar, carregador sol...",UO021,09/04/2024,31/05/2024,maxwelfreitas
